In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from keybert import KeyBERT
import yake
import string

In [2]:
sds_text = pd.read_csv('../data/sds_text.csv')

<IPython.core.display.Javascript object>

In [3]:
sds_text.head(5)

,Unnamed: 0,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_day,host_episode,speaker,episode_split_text
0,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,This is episode number one with ex-chemical e...
1,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,"Hey guys, welcome to the Podcast. I’ve got Ru..."
2,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,Thank you! Thanks for having me over. I’m doi...
3,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,Awesome. It’s great to hear you and for those...
4,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,"Yeah sure. So, I’m the senior manager of anal..."


In [4]:
sds_text = sds_text.set_index('Unnamed: 0')

In [5]:
sds_processed = pd.DataFrame(sds_text.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_date'])['episode_split_text'].agg(lambda x: ' '.join(x))).reset_index()

<IPython.core.display.Javascript object>

In [6]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [7]:
sds_processed['episode_split_text'] = sds_processed['episode_split_text'].apply(processed_text)

In [8]:
sds_processed

,episode_number,episode_name,length_episode,context_episode,guest_name,host_episode,episode_date,episode_split_text
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,Kirril Eremenko,"Sep 10, 2016",this is episode number one with exchemical en...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning Data Science,Hadelin de Ponteves,Kirril Eremenko,"Sep 14, 2016",this is session number two with machine learn...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning R Programming Data Science,Dr. Wilson Pok,Kirril Eremenko,"Sep 25, 2016",this is episode number three with nanophysics ...
3,4,"Data and Strategy, three Pillars of Research ...",60,Business Data Science,Brendan Hogan,Kirril Eremenko,"Oct 02, 2016",this is episode four with business strategy e...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirril Eremenko,"Oct 09, 2016",this is episode number five with forensics inv...
...,...,...,...,...,...,...,...,...
675,679,"The A.I. and Machine Learning Landscape, with...",94,Business Data Science Artificial Intelligence,George Mathew,Jon Krohn,"May 16, 2023",this is episode number with george matthew ma...
676,680,Automating Industrial Machines with Data Scie...,30,Business Data Science,Allegra Alessi,Jon Krohn,"May 19, 2023",this is episode number with allegra alessi io...
677,681,XGBoost,72,Machine Learning Data Science Python,Matt Harrison,Jon Krohn,"May 23, 2023",this is episode number with matt harrison man...
678,682,"Business Intelligence Tools, with Mico Yuk",28,Business Data Science,Mico Yuk,Jon Krohn,"May 26, 2023",this is episode number with mico yuk host of ...


In [10]:
# Function to generate 100 keywords per episode using Yake library

def yake_keyword_extractor(doc):
    global sds_processed

    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n=max_ngram_size, dedupLim=deduplication_threshold, windowsSize=windowSize, top=numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)

        # Create separate columns for each keyword
        for j, keyword in enumerate(keywords):
            column_name = f'episode_keyword{j+1}'
            sds_processed.at[i, column_name] = keyword[0]  # Store the keyword value in the respective column

In [ ]:
yake_keyword_extractor(sds_processed['episode_split_text'])

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_12016/3233365722.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sds_processed.at[i, column_name] = keyword[0]  # Store the keyword value in the respective column
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_12016/3233365722.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sds_processed.at[i, column_name] = keyword[0]  # Store the keyword value in the respective column
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_12016/3233365722.py:1

In [ ]:
sds_processed

In [ ]:
sds_1.shape

In [ ]:
def yake_keyword_extractor(doc):
    global sds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[20]

In [ ]:
sds_processed.info()

In [ ]:
#sds_processed.to_csv('../data/sds_yake_keywords.csv')

In [ ]:
#sds_keywords = pd.read_csv('../data/sds_yake_keywords.csv')

In [ ]:
sds_1 = sds_processed[sds_processed['episode_number'] == 1]

In [ ]:
sds_1

In [ ]:
sds_1.info()

In [ ]:
sds_1['episode_keywords'].loc[0]

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = None

In [ ]:
sds_processed['keywords'] = None

In [ ]:
def tuple_extractor(row):
    words = [t[0] for t in row]
    return words

In [ ]:
sds_processed['keywords'] = sds_processed['episode_keywords'].apply(tuple_extractor)

In [ ]:
print(sds_processed['keywords'].loc[677])

In [ ]:
sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1.loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(word_tuples):
    global sds_1
    words = [t[0] for t in word_tuples]
    return words

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(tuple):
    global sds_keywords
    words = []
    #for i, episode_text in doc.iteritems():
    for i, word_tuple in tuple.iteritems():
        words = [t[0] for t in word_tuple]
        sds_keywords['keywords'] = words
    
            

In [ ]:
sds_keywords['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_keywords

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1

In [ ]:
[t[0] for t in sds_1['episode_keywords']]

In [ ]:
sds_processed['episode_keywords'] = [t[0] for t in sds_processed['episode_keywords']]

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[678]

In [ ]:
doca = sds_processed['episode_split_text'].loc[1]
docb = sds_processed['episode_split_text'].loc[150]
docc = sds_processed['episode_split_text'].loc[400]
docd = sds_processed['episode_split_text'].loc[679]

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 1), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.7)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
kw_model = KeyBERT()
stopwords = list(STOP_WORDS)
keywords = kw_model.extract_keywords(doca, keyphrase_ngram_range=(1, 2), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.8)



In [ ]:
keywords

In [ ]:
keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
len(keywords)

In [ ]:
keywords

In [ ]:
#sds_processed['episode_split_text'][1:5].apply(keyword_extractor)

In [ ]:
# https://maartengr.github.io/BERTopic/index.html